# INTERSECTION MANAGEMENT

### Loading Libraries

In [1]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np

### Loading Dataset

In [2]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

### Setting up GUI

In [3]:
root = tk.Tk()
root.title("Select Four Images")
root.geometry("800x400")

image_paths = ["", "", "", ""]
vehicle_densities = [0, 0, 0, 0]

def select_image(index):
    file_path = filedialog.askopenfilename(
        filetypes=[("Image files", "*.jpg *.jpeg *.png")],
        title=f"Select Image {index + 1}"
    )
    if file_path:
        image_paths[index] = file_path
        update_image_display(index, file_path)
        vehicle_count = detect_vehicles(file_path)
        vehicle_densities[index] = vehicle_count
        print(f"Lane {index + 1} Vehicle Density: {vehicle_count}")

def update_image_display(index, file_path):
    img = Image.open(file_path)
    img.thumbnail((150, 150))
    img_tk = ImageTk.PhotoImage(img)
    image_labels[index].config(image=img_tk)
    image_labels[index].image = img_tk

def detect_vehicles(image_path):
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 2:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

    vehicle_count = len(indices)
    return vehicle_count

def print_image_paths_and_densities():
    for i, path in enumerate(image_paths):
        print(f"Lane {i + 1} - Image Path: {path}, Vehicle Density: {vehicle_densities[i]}")
    print()

image_labels = []
for i in range(4):
    label = tk.Label(root, text=f"Image {i + 1}", relief="solid", width=20, height=10)
    label.grid(row=1, column=i, padx=10, pady=10)
    image_labels.append(label)

buttons = []
for i in range(4):
    button = ttk.Button(root, text=f"Select Image {i + 1}", command=lambda idx=i: select_image(idx))
    button.grid(row=2, column=i, padx=10, pady=10)
    buttons.append(button)

print_button = ttk.Button(root, text="Print Image Paths and Densities", command=print_image_paths_and_densities)
print_button.grid(row=3, column=0, columnspan=4, pady=20)

root.mainloop()

print("Vehicle Densities Array:", vehicle_densities)

Lane 1 Vehicle Density: 25
Lane 2 Vehicle Density: 10
Lane 3 Vehicle Density: 27
Lane 4 Vehicle Density: 21
Lane 1 - Image Path: C:/Users/Vishal V/Major Project/Mini Project/Untitled Folder 3/peak-hour-traffic.png, Vehicle Density: 25
Lane 2 - Image Path: C:/Users/Vishal V/Major Project/Mini Project/Untitled Folder 3/traffic test 1.png, Vehicle Density: 10
Lane 3 - Image Path: C:/Users/Vishal V/Major Project/Mini Project/Untitled Folder 3/trafficJamHyderabad-1.png, Vehicle Density: 27
Lane 4 - Image Path: C:/Users/Vishal V/Major Project/Mini Project/Untitled Folder 3/Untitled-design-2023-09-16T190808.328.png, Vehicle Density: 21



KeyboardInterrupt: 

### Density Function Misc. Code

In [4]:
densities = vehicle_densities

densities

green_time = [0, 0, 0, 0]
for i in range(len(densities)):
    green_time[i] = 2 * densities[i]

time = [0, 0, 0, 0]  # Initialize working times array
wait_time = [0, 0, 0, 0]
priority = [1, 0, 0, 0]

max_green = 30
max_wait = 80
min_green = 20

iteration = 0

### Density Calculation and Iterations

In [5]:
while any(densities):  # Continue until all densities are cleared
    iteration += 1
    print(f"Iteration {iteration}")

    for i in range(4):
        if wait_time[i] > max_wait:
            priority[i] = 10
        else:
            priority[i] = densities[i] if densities[i] > 0 else 0

    if any(wait > max_wait for wait in wait_time):
        high_priority_lane = wait_time.index(max(wait_time))
    else:
        max_priority = max(priority)
        high_priority_lane = priority.index(max_priority)

    actual_green_time = min(green_time[high_priority_lane], max_green)
    actual_green_time = max(actual_green_time, min_green)

    vehicles_passed = actual_green_time // 2
    densities[high_priority_lane] = max(0, densities[high_priority_lane] - vehicles_passed)
    green_time[high_priority_lane] = max(0, green_time[high_priority_lane] - actual_green_time)

    for i in range(4):
        if i == high_priority_lane:
            wait_time[i] = 0
        else:
            wait_time[i] += actual_green_time

    priority[high_priority_lane] = 0

    print(f"Lanes status after processing lane {high_priority_lane + 1}:")
    for k in range(4):
        print(f"Lane {k + 1}: Density = {densities[k]}, Green time = {green_time[k]}, Wait time = {wait_time[k]}")

    for i in range(4):
        if densities[i] > 0:
            priority[i] = densities[i]
        else:
            priority[i] = 0

    if all(density == 0 for density in densities):
        print("All lanes cleared.")
        break

Iteration 1
Lanes status after processing lane 3:
Lane 1: Density = 25, Green time = 50, Wait time = 30
Lane 2: Density = 10, Green time = 20, Wait time = 30
Lane 3: Density = 12, Green time = 24, Wait time = 0
Lane 4: Density = 21, Green time = 42, Wait time = 30
Iteration 2
Lanes status after processing lane 1:
Lane 1: Density = 10, Green time = 20, Wait time = 0
Lane 2: Density = 10, Green time = 20, Wait time = 60
Lane 3: Density = 12, Green time = 24, Wait time = 30
Lane 4: Density = 21, Green time = 42, Wait time = 60
Iteration 3
Lanes status after processing lane 4:
Lane 1: Density = 10, Green time = 20, Wait time = 30
Lane 2: Density = 10, Green time = 20, Wait time = 90
Lane 3: Density = 12, Green time = 24, Wait time = 60
Lane 4: Density = 6, Green time = 12, Wait time = 0
Iteration 4
Lanes status after processing lane 2:
Lane 1: Density = 10, Green time = 20, Wait time = 50
Lane 2: Density = 0, Green time = 0, Wait time = 0
Lane 3: Density = 12, Green time = 24, Wait time = 